# *Sprawozdanie ćw. 14 - WÓJCIK (198212) JAKIEL (197733)*

## *Wprowadzenie :*
Niniejsze sprawozdanie opisuje implementację symulatora układu dynamicznego opisanego transmitancją $G(s)$ wraz z regulatorem $G_c(s)$ w konfiguracji z ujemnym sprzężeniem zwrotnym. Symulator umożliwia analizę odpowiedzi czasowej układu na różne rodzaje sygnałów wejściowych (prostokątny o skończonym czasie trwania, trójkątny, harmoniczny) oraz pozwala na zmianę parametrów transmitancji i sygnałów wejściowych. W sprawozdaniu szczegółowo opisano proces uzyskania transmitancji układu zamkniętego oraz zastosowaną metodę numeryczną – metodę Eulera – do rozwiązania równań różniczkowych opisujących dynamikę układu.

## 1. Rozwinięcie opisu uzyskania mianownika transmitancji wypadkowej ze sprzężeniem zwrotnym

W analizowanym układzie mamy do czynienia z systemem sterowania w konfiguracji z ujemnym sprzężeniem zwrotnym. Składa się on z dwóch głównych bloków opisanych transmitancjami: 
- członu głównego $G(s)$ oraz 
- członu regulatora $G_c(s)$.

Transmitancje te opisują dynamiczne właściwości poszczególnych komponentów w dziedzinie zespolonej zmiennej \( s \), będącej zmienną Laplace’a.

Transmitancja członu głównego ma postać:

$$
G(s) = \frac{a_1s + a_0}{b_2s^2 + b_1s + b_0}
$$

Natomiast transmitancja regulatora ma postać:

$$
G_c(s) = \frac{c_2s^2 + c_1s + c_0}{d_2s^2 + d_1s + d_0}
$$

Transmitancja otwartej pętli układu, czyli wypadkowa transmitancja połączonych szeregowo członu głównego i regulatora, jest iloczynem transmitancji składowych:

$$
G_o(s) = G(s) \cdot G_c(s) = \left( \frac{a_1s + a_0}{b_2s^2 + b_1s + b_0} \right) \cdot \left( \frac{c_2s^2 + c_1s + c_0}{d_2s^2 + d_1s + d_0} \right)
$$

Po wykonaniu mnożenia wielomianów w liczniku i mianowniku otrzymujemy:

$$
G_o(s) = \frac{(a_1s + a_0)(c_2s^2 + c_1s + c_0)}{(b_2s^2 + b_1s + b_0)(d_2s^2 + d_1s + d_0)} = \frac{(a_1c_2)s^3 + (a_1c_1 + a_0c_2)s^2 + (a_1c_0 + a_0c_1)s + (a_0c_0)}{(b_2d_2)s^4 + (b_2d_1 + b_1d_2)s^3 + (b_2d_0 + b_1d_1 + b_0d_2)s^2 + (b_1d_0 + b_0d_1)s + (b_0d_0)}
$$

Oznaczając licznik transmitancji otwartej pętli jako $L(s)$ i mianownik jako $M(s)$:

$$
L(s) = (a_1c_2)s^3 + (a_1c_1 + a_0c_2)s^2 + (a_1c_0 + a_0c_1)s + (a_0c_0)
$$

$$
M(s) = (b_2d_2)s^4 + (b_2d_1 + b_1d_2)s^3 + (b_2d_0 + b_1d_1 + b_0d_2)s^2 + (b_1d_0 + b_0d_1)s + (b_0d_0)
$$

W układzie z ujemnym sprzężeniem zwrotnym jednostkowym, sygnał wyjściowy jest odejmowany od sygnału wejściowego, a różnica stanowi sygnał sterujący dla otwartej pętli. Transmitancja układu zamkniętego $G_z(s)$, która opisuje zależność między sygnałem wyjściowym a wejściowym całego układu ze sprzężeniem zwrotnym, dana jest ogólnym wzorem:

$$
G_z(s) = \frac{G_o(s)}{1 + G_o(s)}
$$

Podstawiając $ G_o(s) = \frac{L(s)}{M(s)} $ do powyższego wzoru, otrzymujemy:

$$
G_z(s) = \frac{\frac{L(s)}{M(s)}}{1 + \frac{L(s)}{M(s)}} = \frac{L(s)}{M(s) + L(s)}
$$

Z tego wyrażenia jasno wynika, że mianownik transmitancji układu zamkniętego $ G_z(s) $, który oznaczamy jako $ D(s) $, jest sumą mianownika transmitancji otwartej pętli $ M(s) $ i licznika tej transmitancji $ L(s) $:

$$
D(s) = M(s) + L(s)
$$

Podstawiając rozwinięte formy $ L(s) $ i $ M(s) $, otrzymujemy:

$$
D(s) = (b_2d_2)s^4 + (b_2d_1 + b_1d_2)s^3 + (b_2d_0 + b_1d_1 + b_0d_2)s^2 + (b_1d_0 + b_0d_1)s + (b_0d_0) + (a_1c_2)s^3 + (a_1c_1 + a_0c_2)s^2 + (a_1c_0 + a_0c_1)s + (a_0c_0)
$$

Po pogrupowaniu wyrazów o tych samych potęgach zmiennej $ s $, otrzymujemy ostateczną postać mianownika wypadkowej transmitancji układu zamkniętego:

$$
D(s) = (b_2d_2)s^4 + (b_2d_1 + b_1d_2 + a_1c_2)s^3 + (b_2d_0 + b_1d_1 + b_0d_2 + a_1c_1 + a_0c_2)s^2 + (b_1d_0 + b_0d_1 + a_1c_0 + a_0c_1)s + (b_0d_0 + a_0c_0)
$$

## 2. Rozwinięcie opisu metody Eulera dla równań różniczkowych

Metoda Eulera jest jedną z najprostszych metod numerycznych służących do przybliżonego rozwiązywania równań różniczkowych zwyczajnych pierwszego rzędu z zadanym warunkiem początkowym. Jej podstawowa idea polega na aproksymacji pochodnej funkcji w danym punkcie za pomocą ilorazu różnicowego, co geometrycznie odpowiada przybliżeniu krzywej rozwiązania przez styczną w tym punkcie.

Rozważmy ogólne równanie różniczkowe pierwszego rzędu:

$$
\frac{dy}{dt} = f(t, y(t))
$$

z warunkiem początkowym $y(t_0) = y_0$. Chcemy znaleźć przybliżoną wartość funkcji $y(t)$ w dyskretnych momentach czasu $t_k = t_0 + k \cdot h$, gdzie $h$ jest krokiem czasowym.

### Metoda Eulera:

W metodzie Eulera, pochodną $\frac{dy}{dt}$ w chwili $t_k$ przybliżamy za pomocą ilorazu różnicowego:


$$
\frac{dy}{dt} \approx \frac{y(t_k + h) - y(t_k)}{h} = \frac{y_k - y_{k-1}}{h}
$$

gdzie $y_k$ oznacza przybliżoną wartość $y(t_k)$, a $y_{k+1}$ oznacza przybliżoną wartość $y(t_{k+1})$.

Podstawiając to przybliżenie do równania różniczkowego, otrzymujemy:

$$
\frac{y_k - y_{k-1}}{h} \approx f(t_k, y_k)
$$

Przekształcając to równanie, otrzymujemy iteracyjny wzór metody Eulera:

$$
y_k = y_{k-1} + h \cdot f(t_k, y_k)
$$

Ten wzór pozwala nam krok po kroku obliczać przybliżone wartości rozwiązania równania różniczkowego, zaczynając od znanego warunku początkowego.

---

### Rozszerzenie na równania różniczkowe wyższych rzędów:

W praktyce często mamy do czynienia z równaniami różniczkowymi wyższych rzędów. Aby móc zastosować do nich metodę Eulera, konieczne jest przekształcenie takiego równania w układ równań różniczkowych pierwszego rzędu.

Rozważmy równanie różniczkowe $n$-tego rzędu:

$$
\frac{d^n y}{dt^n} = g\left(t, y, \frac{dy}{dt}, \ldots, \frac{d^{n-1}y}{dt^{n-1}}\right)
$$

Wprowadzamy nowe zmienne stanu:

$$
x_1(t) = y(t)
$$
$$
x_2(t) = \frac{dy}{dt} = \frac{dx_1}{dt}
$$
$$
x_3(t) = \frac{d^2 y}{dt^2} = \frac{dx_2}{dt}
$$
$$
\vdots
$$
$$
x_n(t) = \frac{d^{n-1} y}{dt^{n-1}} = \frac{dx_{n-1}}{dt}
$$

Wtedy równanie $n$-tego rzędu jest równoważne układowi $n$ równań różniczkowych pierwszego rzędu:

$$
\frac{dx_1}{dt} = x_2
$$
$$
\frac{dx_2}{dt} = x_3
$$
$$
\vdots
$$
$$
\frac{dx_{n-1}}{dt} = x_n
$$
$$
\frac{dx_n}{dt} = g(t, x_1, x_2, \ldots, x_n)
$$

Do każdego z tych równań możemy zastosować metodę Eulera:

$$
x_{1,k} = x_{1,{k-1}} + h \cdot x_{2,k}
$$
$$
x_{2,k} = x_{2,{k-1}} + h \cdot x_{3,k}
$$
$$
\vdots
$$
$$
x_{n-1,k} = x_{n-1,{k-1}} + h \cdot x_{n,k}
$$
$$
x_{n,k} = x_{n,{k-1}} + h \cdot g(t_k, x_{1,k}, x_{2,k}, \ldots, x_{n,k})
$$

## 3. Implementacja numeryczna – odpowiedź układu metodą Eulera

W tej części sprawozdania przedstawiona została implementacja metody Eulera do rozwiązania równania różniczkowego wyższego rzędu wynikającego z transmitancji układu.

### Transmitancja układu

Rozważamy układ, którego transmitancja otwarta $ G_o(s) $ wynika z dwóch połączonych szeregowo transmitancji $ G(s) $ oraz $ G_c(s) $:

$$
G(s) = \frac{a_1 s + a_0}{b_2 s^2 + b_1 s + b_0}, \quad G_c(s) = \frac{c_2 s^2 + c_1 s + c_0}{d_2 s^2 + d_1 s + d_0}
$$

Transmitancja układu zamkniętego z ujemnym sprzężeniem zwrotnym:

$$
G_z(s) = \frac{G(s) G_c(s)}{1 + G(s) G_c(s)} = \frac{(a_1c_2)s^3 + (a_1c_1 + a_0c_2)s^2 + (a_1c_0 + a_0c_1)s + (a_0c_0)}{(b_2d_2)s^4 + (b_2d_1 + b_1d_2 + a_1c_2)s^3 + (b_2d_0 + b_1d_1 + b_0d_2 + a_1c_1 + a_0c_2)s^2 + (b_1d_0 + b_0d_1 + a_1c_0 + a_0c_1)s + (b_0d_0 + a_0c_0)} = \frac{L(s)}{D(s)}
$$
Gdzie:

$$
l_3 = a_1 c_2
$$

$$
l_2 = a_1 c_1 + a_0 c_2
$$

$$
l_1 = a_1 c_0 + a_0 c_1
$$

$$
l_0 = a_0 c_0
$$

oraz:

$$
d_4 = b_2 d_2
$$

$$
d_3 = b_2 d_1 + b_1 d_2 + a_1 c_2
$$

$$
d_2 = b_2 d_0 + b_1 d_1 + b_0 d_2 + a_1 c_1 + a_0 c_2
$$

$$
d_1 = b_1 d_0 + b_0 d_1 + a_1 c_0 + a_0 c_1
$$

$$
d_0 = b_0 d_0 + a_0 c_0
$$
    
Z relacji:

$$
Y(s) = G_z(s) \cdot U(s)
$$

przechodzimy do postaci równania różniczkowego:

$$
D(s)Y(s) = L(s)U(s)
$$

Rozwijając obie strony równania, otrzymujemy:

$$
\left( d_4 s^4 + d_3 s^3 + d_2 s^2 + d_1 s + d_0 \right) Y(s) = \left( l_3 s^3 + l_2 s^2 + l_1 s + l_0 \right) U(s)
$$

czyli:

$$
d_4 s^4 Y(s) = \left( l_3 s^3 + l_2 s^2 + l_1 s + l_0 \right) U(s) - \left( d_3 s^3 + d_2 s^2 + d_1 s + d_0 \right) Y(s)
$$

Dzieląc obie strony przez $ d_4 $, otrzymujemy ostatecznie:

$$
s^4 Y(s) = \frac{1}{d_4} \left[ \left( l_3 s^3 + l_2 s^2 + l_1 s + l_0 \right) U(s) - \left( d_3 s^3 + d_2 s^2 + d_1 s + d_0 \right) Y(s) \right]
$$

W dziedzinie czasu:

$$
d_4 \frac{d^4 y}{dt^4} + d_3 \frac{d^3 y}{dt^3} + d_2 \frac{d^2 y}{dt^2} + d_1 \frac{dy}{dt} + d_0 y = l_3 \frac{d^3 u}{dt^3} + l_2 \frac{d^2 u}{dt^2} + l_1 \frac{du}{dt} + l_0 u
$$

Wyodrębniając $ \frac{d^4 y}{dt^4} $, otrzymujemy:

$$
\frac{d^4 y}{dt^4} = \frac{1}{d_4} \left( l_3 \frac{d^3 u}{dt^3} + l_2 \frac{d^2 u}{dt^2} + l_1 \frac{du}{dt} + l_0 u - d_3 \frac{d^3 y}{dt^3} - d_2 \frac{d^2 y}{dt^2} - d_1 \frac{dy}{dt} - d_0 y \right)
$$

### Numeryczna implementacja równania

Zanim nastąpi obliczenie odpowiedzi układu, konieczne jest wyznaczenie pochodnych sygnału wejściowego $ u(t) $, zgodnie z rzędem układu i jego transmitancją:
$$
d u_{k}= \frac{u_{k} - u_{k-1}}{h}
$$

$$
d^2 u_{k}= \frac{d u_{k} - d u_{k-1}}{h}
$$

$$
d^3 u_{k}= \frac{d^2 u_{k} - d^2 u_{k-1}}{h}
$$

Wartości te są następnie wykorzystywane do obliczenia pochodnych sygnału wyjściowego $ y(t) $ przy pomocy zależności wynikającej z równań różniczkowych układu:

$$
y^{(4)} = \frac{1}{d_4} \left( l_3 u^{(3)} + l_2 u^{(2)} + l_1 \dot{u} + l_0 u - d_3 y^{(3)} - d_2 \ddot{y} - d_1 \dot{y} - d_0 y \right)
$$

oraz:

$$
y_{k} = y_{k-1} + h \cdot \dot{y}_k
$$
$$
\dot{y}_{k} = \dot{y}_{k-1} + h \cdot \ddot{y}_k
$$
$$
\ddot{y}_{k} = \ddot{y}_{k-1} + h \cdot y^{(3)}_k 
$$
$$
y^{(3)}_{k} = y^{(3)}_{k-1} + h \cdot y^{(4)}_k
$$

### Wyniki

Odpowiedź układu została zobrazowana graficznie przy użyciu biblioteki `matplotlib`, gdzie porównano pobudzenie $ u(t) $ z odpowiedzią układu $ y(t) $. Analiza wykresu pozwala na ocenę poprawności działania metody Eulera oraz charakterystyki dynamicznej układu.

## 4. Uogólnienie implementacji dla układów różnych rzędów

Przedstawiona wcześniej implementacja skupiała się na przypadku, gdy transmitancja wypadkowa układu zamkniętego $G_z(s)$ była rzędu czwartego. W praktyce często spotykamy się z układami o niższych rzędach. Zaktualizowany kod w języku Python rozszerza funkcjonalność o możliwość symulacji odpowiedzi układów rzędu pierwszego, drugiego, trzeciego oraz czwartego.

### Modyfikacja równania różniczkowego w zależności od rzędu układu

Ogólną postać transmitancji układu zamkniętego można zapisać jako:

$$G_z(s) = \frac{L(s)}{D(s)} = \frac{l_m s^m + l_{m-1} s^{m-1} + \dots + l_1 s + l_0}{d_n s^n + d_{n-1} s^{n-1} + \dots + d_1 s + d_0}$$

gdzie $n$ jest rzędem mianownika (co definiuje rząd układu), a $m$ jest rzędem licznika, przy czym zazwyczaj $m \le n$.

Odpowiadające temu równanie różniczkowe ma postać:

$$d_n \frac{d^n y}{dt^n} + d_{n-1} \frac{d^{n-1} y}{dt^{n-1}} + \dots + d_1 \frac{dy}{dt} + d_0 y = l_m \frac{d^m u}{dt^m} + l_{m-1} \frac{d^{m-1} u}{dt^{m-1}} + \dots + l_1 \frac{du}{dt} + l_0 u$$

Aby zastosować metodę Eulera, wyodrębniamy najwyższą pochodną sygnału wyjściowego $y(t)$:

$$\frac{d^n y}{dt^n} = \frac{1}{d_n} \left( \sum_{i=0}^{m} l_i \frac{d^i u}{dt^i} - \sum_{j=0}^{n-1} d_j \frac{d^j y}{dt^j} \right)$$

W kodzie Python, zmienna `rzad` odpowiada wartości $n$. W zależności od wartości `rzad`, najwyższa obliczana pochodna sygnału wyjściowego $y(t)$ oraz liczba uwzględnianych pochodnych sygnału wejściowego $u(t)$ i wyjściowego $y(t)$ w równaniu ulegają zmianie. Współczynniki transmitancji `num` odpowiadają $l_i$, a `den` odpowiadają $d_i$. Przyjmujemy, że `den[0]` to współczynnik $d_n$ (przy najwyższej potędze $s$ w mianowniku), a `num[0]` to współczynnik $l_m$ (przy najwyższej potędze $s$ w liczniku dla danego rzędu układu).

Poniżej przedstawiono, jak równanie na najwyższą pochodną wyjścia oraz iteracje metody Eulera upraszczają się dla różnych wartości `rzad`.

#### Przypadek `rzad = 4` (rząd czwarty)

Równanie na czwartą pochodną wyjścia:
$$\frac{d^4 y}{dt^4} = \frac{1}{\text{den}[0]} \left( \text{num}[0] \frac{d^3 u}{dt^3} + \text{num}[1] \frac{d^2 u}{dt^2} + \text{num}[2] \frac{du}{dt} + \text{num}[3] u - \text{den}[1] \frac{d^3 y}{dt^3} - \text{den}[2] \frac{d^2 y}{dt^2} - \text{den}[3] \frac{dy}{dt} - \text{den}[4] y \right)$$
$$y^{(4)}_k \text{ (obliczone z powyższego wzoru)}$$
$$y^{(3)}_{k} = y^{(3)}_{k-1} + T_s \cdot y^{(4)}_k$$
$$\ddot{y}_{k} = \ddot{y}_{k-1} + T_s \cdot y^{(3)}_k$$
$$\dot{y}_{k} = \dot{y}_{k-1} + T_s \cdot \ddot{y}_k$$
$$y_{k} = y_{k-1} + T_s \cdot \dot{y}_k$$

#### Przypadek `rzad = 3` (rząd trzeci)

Równanie na trzecią pochodną wyjścia:
$$\frac{d^3 y}{dt^3} = \frac{1}{\text{den}[0]} \left( \text{num}[0] \frac{d^3 u}{dt^3} + \text{num}[1] \frac{d^2 u}{dt^2} + \text{num}[2] \frac{du}{dt} + \text{num}[3] u - \text{den}[1] \frac{d^2 y}{dt^2} - \text{den}[2] \frac{dy}{dt} - \text{den}[3] y \right)$$
$$y^{(3)}_k \text{ (obliczone z powyższego wzoru)}$$
$$\ddot{y}_{k} = \ddot{y}_{k-1} + T_s \cdot y^{(3)}_k$$
$$\dot{y}_{k} = \dot{y}_{k-1} + T_s \cdot \ddot{y}_k$$
$$y_{k} = y_{k-1} + T_s \cdot \dot{y}_k$$

#### Przypadek `rzad = 2` (rząd drugi)

Równanie na drugą pochodną wyjścia:
$$\frac{d^2 y}{dt^2} = \frac{1}{\text{den}[0]} \left( \text{num}[0] \frac{d^3 u}{dt^3} + \text{num}[1] \frac{d^2 u}{dt^2} + \text{num}[2] \frac{du}{dt} + \text{num}[3] u - \text{den}[1] \frac{dy}{dt} - \text{den}[2] y \right)$$
$$\ddot{y}_k \text{ (obliczone z powyższego wzoru)}$$
$$\dot{y}_{k} = \dot{y}_{k-1} + T_s \cdot \ddot{y}_k$$
$$y_{k} = y_{k-1} + T_s \cdot \dot{y}_k$$

#### Przypadek `rzad = 1` (rząd pierwszy)

Równanie na pierwszą pochodną wyjścia:
$$\frac{dy}{dt} = \frac{1}{\text{den}[0]} \left( \text{num}[0] \frac{d^3 u}{dt^3} + \text{num}[1] \frac{d^2 u}{dt^2} + \text{num}[2] \frac{du}{dt} + \text{num}[3] u - \text{den}[1] y \right)$$
$$\dot{y}_k \text{ (obliczone z powyższego wzoru)}$$
$$y_{k} = y_{k-1} + T_s \cdot \dot{y}_k$$

---

## Podsumowanie oraz Wnioski:

- Transformata Laplace’a okazała się skutecznym narzędziem do analizy i uproszczenia układów równań różniczkowych poprzez przejście do dziedziny operatorowej. Umożliwiła algebraiczne rozwiązanie równań, co znacząco ułatwiło dalszą analizę dynamiki układu.

- Opracowano i zaimplementowano metodę uzysku mianownika transmitancji wypadkowej $G(s)$ dla układu dwóch sprzężonych elementów. Wyprowadzenie wzorów na transmitancję $G(s)$ pozwoliło na dokładne modelowanie układu.

- Metoda Eulera okazała się prostą i skuteczną metodą numeryczną do przybliżonego rozwiązywania równań różniczkowych zwyczajnych. Jej implementacja jest intuicyjna i efektywna dla prostych układów, choć wymaga uwagi przy doborze kroku całkowania.

- Przeprowadzona implementacja numeryczna w środowisku obliczeniowym, w tym w Pythonie, umożliwiła symulację zachowania badanego układu dynamicznego. Uzyskane wyniki potwierdziły poprawność przyjętego modelu oraz wybranej metody obliczeniowej.

- Uogólnienie implementacji pokazało, że metoda Eulera może być z powodzeniem stosowana do układów pierwszego, drugiego oraz wyższych rzędów, pod warunkiem odpowiedniego przekształcenia równań do postaci pierwszorzędowej. Potwierdzono to w symulacjach układów o różnej strukturze.

- Dokładność metody Eulera jest silnie zależna od wielkości kroku całkowania. Mniejsze kroki prowadzą do większej precyzji, ale zwiększają czas obliczeń, co uwidoczniło się w symulacjach bardziej złożonych układów.

- W przypadku układów wyższego rzędu i bardziej złożonej struktury kluczowe okazało się odpowiednie dobranie kroku całkowania oraz wstępne przekształcenie równań do postaci kanonicznej, co zapewniło stabilność i dokładność obliczeń.

- Połączenie transformaty Laplace’a z numeryczną metodą Eulera pozwoliło na kompleksową analizę układów dynamicznych. Wyniki symulacji były zgodne z przewidywaniami teoretycznymi, a opracowane narzędzia obliczeniowe w Pythonie umożliwiają dalsze badania i rozszerzenie na bardziej złożone systemy. Odpowiedni dobór parametrów, takich jak krok całkowania, jest kluczowy dla uzyskania wiarygodnych wyników.

